In [1]:
import pandas as pd
import toml
import h5py
from pathlib import Path
import functions
import geopandas as gpd
import os

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.options.display.float_format = '{:0,.0f}'.format

output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]

In [2]:
df = functions.process_network_summary()

df = df[df['county']!="Outside Region"]

# network 
df = df.copy()
df['medium_truck_vmt'] = df['@mveh']*df['length']
df['heavy_truck_vmt'] = df['@hveh']*df['length']
df['truck_vmt'] = df['heavy_truck_vmt']+df['medium_truck_vmt']

# add order for county
df["county"] = pd.Categorical(df["county"], categories = ['King', 'Snohomish', 'Pierce', 'Kitsap'], ordered=True)

df['Road Type'] = df['@fgts'].map({
    0: 'Non-Truck Route',
    1: 'FGTS 1',
    2: 'FGTS 2'
})

## Truck Miles Traveled by County

In [3]:
tab = df[['county','truck_vmt','medium_truck_vmt','heavy_truck_vmt']].groupby('county', observed=False).sum()[['medium_truck_vmt','heavy_truck_vmt','truck_vmt']]
truck_col_map = {'truck_vmt': 'All Trucks',
                 'medium_truck_vmt': 'Medium Trucks',
                'heavy_truck_vmt': 'Heavy Trucks'}
tab = tab.rename(columns=truck_col_map)
tab.loc['Total',:] = tab.sum()
tab

,Medium Trucks,Heavy Trucks,All Trucks
county,,,
King,"1,956,074","1,417,574","3,373,647"
Snohomish,"551,474","534,242","1,085,716"
Pierce,"549,796","539,816","1,089,612"
Kitsap,"145,278","90,379","235,657"
Total,"3,202,621","2,582,011","5,784,632"


## Miles Traveled on Truck Routes by Vehicle Type

Major Truck Routes include two tiers on the Freight and Goods Transportation System (FGTS)

- T-1: More than 10 million tons per year

- T-2: 4 million to 10 million tons per year

In [4]:
# Travel on Truck Routes

df_vmt = df[['VMT','heavy_truck_vmt','medium_truck_vmt','Road Type']].groupby('Road Type').sum()
df_vmt['Passenger'] = df_vmt['VMT'] - df_vmt['heavy_truck_vmt'] - df_vmt['medium_truck_vmt']
df_vmt = df_vmt.rename(columns={'heavy_truck_vmt': 'Heavy Truck',
                             'medium_truck_vmt': 'Medium Truck',})
df1 = df_vmt.loc[['FGTS 1', 'FGTS 2']]
df1 = df1[['Passenger', 'Medium Truck', 'Heavy Truck']]
df1

,Passenger,Medium Truck,Heavy Truck
Road Type,,,
FGTS 1,"31,090,920","1,778,551","2,086,019"
FGTS 2,"11,378,013","486,324","172,121"


In [5]:
df_vmt = df_vmt[['Passenger', 'Medium Truck', 'Heavy Truck']]
df_vmt.loc['Major Truck Routes (FGTS 1 + FGTS 2)',:] = df_vmt.loc['FGTS 1',:] + df_vmt.loc['FGTS 2',:]
df2 = df_vmt.loc[['Non-Truck Route','Major Truck Routes (FGTS 1 + FGTS 2)']]
df2

,Passenger,Medium Truck,Heavy Truck
Road Type,,,
Non-Truck Route,"33,697,964","937,747","323,871"
Major Truck Routes (FGTS 1 + FGTS 2),"42,468,934","2,264,875","2,258,140"


In [6]:
# Show percentage of VMT by vehicle type
pd.options.display.float_format = '{:0,.1%}'.format
df1_pct = df1.div(df1.sum(axis=0), axis=1)
df1_pct

,Passenger,Medium Truck,Heavy Truck
Road Type,,,
FGTS 1,73.2%,78.5%,92.4%
FGTS 2,26.8%,21.5%,7.6%


In [7]:
df2_pct = df2.div(df2.sum(axis=0), axis=1)
df2_pct

,Passenger,Medium Truck,Heavy Truck
Road Type,,,
Non-Truck Route,44.2%,29.3%,12.5%
Major Truck Routes (FGTS 1 + FGTS 2),55.8%,70.7%,87.5%


## Vehicle Trips

In [8]:
truck_h5 = h5py.File(Path(output_path,'trucks','truck_trips.h5'), 'r')

total_med_trips = 0
total_hvy_trips = 0
for tod in truck_h5.keys():
    total_med_trips += truck_h5[tod]['mf'+tod+'_medtrk_trips'][:].sum()
    total_hvy_trips += truck_h5[tod]['mf'+tod+'_hvytrk_trips'][:].sum()

_df_trips = pd.DataFrame({'Trips': [total_hvy_trips, total_med_trips]}, index=['Heavy Trucks', 'Medium Trucks'])

In [9]:
skim_h5 = h5py.File('../../../../inputs/model/daysim/roster/7to8.h5', 'r')

wt_hvy = (truck_h5['7to8']['mf7to8_hvytrk_trips'][0:3751, 0:3751] * (skim_h5['Skims']['heavy_truckd'][:]/100.0)[0:3751, 0:3751]).sum()
hvy_dist = wt_hvy/truck_h5['7to8']['mf7to8_hvytrk_trips'][0:3751, 0:3751].sum()

wt_med = (truck_h5['7to8']['mf7to8_medtrk_trips'][0:3751, 0:3751] * (skim_h5['Skims']['medium_truckd'][:]/100.0)[0:3751, 0:3751]).sum()
med_dist = wt_med/truck_h5['7to8']['mf7to8_medtrk_trips'][0:3751, 0:3751].sum()

tot_dist = (wt_hvy+wt_med)/(truck_h5['7to8']['mf7to8_medtrk_trips'][0:3751, 0:3751].sum()+truck_h5['7to8']['mf7to8_hvytrk_trips'][0:3751, 0:3751].sum())

_df_dist = pd.DataFrame({'Average Distance (mi)': [hvy_dist, med_dist]}, index=['Heavy Trucks', 'Medium Trucks'])

In [10]:
pd.options.display.float_format = '{:0,.0f}'.format
df_combined = pd.concat([_df_trips, _df_dist], axis=1)
df_combined.loc['All Trucks',:] = df_combined.sum()
df_combined.loc['All Trucks','Average Distance (mi)'] = tot_dist
df_combined['Average Distance (mi)'] = df_combined['Average Distance (mi)'].map('{:.1f}'.format)
df_combined

,Trips,Average Distance (mi)
Heavy Trucks,"134,623",38.8
Medium Trucks,"318,630",14.9
All Trucks,"453,253",22.0


## Truck Delay

- Delay Hours is total delay for all trucks on an average weekday

- Annual Avg Driver Delay is the total average hourly delay for a driver in a year 

In [11]:
pd.options.display.float_format = '{:0,.0f}'.format
df_delay = pd.read_csv(Path(output_path, 'network', 'delay_user_class.csv'))
df_delay = pd.DataFrame(df_delay[['@hveh','@mveh']].sum(), columns=['Delay Hours'])
df_delay.rename(index={'@mveh': 'Medium Trucks',
               '@hveh': 'Heavy Trucks'}, inplace=True)
df_delay = df_delay.merge(df_combined.loc[["Heavy Trucks","Medium Trucks"]], left_index=True, right_index=True)[['Delay Hours','Trips']]
df_delay['Annual Avg Driver Delay'] = (df_delay['Delay Hours']/df_delay['Trips'])*summary_config['weekday_to_annual']
df_delay['Annual Avg Driver Delay'] = df_delay['Annual Avg Driver Delay'].map('{:.1f}'.format)
df_delay[['Delay Hours','Annual Avg Driver Delay']]


,Delay Hours,Annual Avg Driver Delay
Heavy Trucks,"6,743",16.0
Medium Trucks,"9,369",9.4


## External Trips

In [12]:
# External-external trips
ext_med_trips = 0
ext_hvy_trips = 0
for tod in truck_h5.keys():
    ext_med_trips += truck_h5[tod]['mf'+tod+'_medtrk_trips'][3700:,3700:].sum()
    ext_hvy_trips += truck_h5[tod]['mf'+tod+'_hvytrk_trips'][3700:,3700:].sum()

df_ext = pd.DataFrame({'External -> External (Pass Through)': [ext_hvy_trips, ext_med_trips]}, index=['Heavy Trucks', 'Medium Trucks'])

# Internal ->  external trips
ext_med_trips = 0
ext_hvy_trips = 0
for tod in truck_h5.keys():
    ext_med_trips += truck_h5[tod]['mf'+tod+'_medtrk_trips'][:3701,3700:].sum()
    ext_hvy_trips += truck_h5[tod]['mf'+tod+'_hvytrk_trips'][:3701,3700:].sum()

df_i_e = pd.DataFrame({'Internal -> External': [ext_hvy_trips, ext_med_trips]}, index=['Heavy Trucks', 'Medium Trucks'])
df_ext = df_ext.merge(df_i_e, left_index=True, right_index=True)

# External ->  internal trips
ext_med_trips = 0
ext_hvy_trips = 0
for tod in truck_h5.keys():
    ext_med_trips += truck_h5[tod]['mf'+tod+'_medtrk_trips'][3700:,:3701].sum()
    ext_hvy_trips += truck_h5[tod]['mf'+tod+'_hvytrk_trips'][3700:,:3701].sum()

df_e_i = pd.DataFrame({'External -> Internal': [ext_hvy_trips, ext_med_trips]}, index=['Heavy Trucks', 'Medium Trucks'])
df_ext = df_ext.merge(df_e_i, left_index=True, right_index=True)

# # Total Trips
# (_df_trips*2) - thru_truck_trips
df_ext['Total'] = df_ext.sum(axis=1)
df_ext = pd.DataFrame(df_ext.loc['Heavy Trucks'])

df_ext.rename(columns={'Heavy Trucks': 'Trips'}, inplace=True)
df_ext

,Trips
External -> External (Pass Through),"4,531"
Internal -> External,"22,236"
External -> Internal,"22,236"
Total,"49,002"


### Miles of Roadway with AM Congestion

In [13]:
time_period_list = ['7to8']
_df = df[df['tod'].isin(time_period_list)]
_df = _df.pivot_table(index='congestion_category',columns='county',
               aggfunc='sum',values='length')
_df = _df/len(time_period_list)
_df.to_clipboard()
_df

C:\Users\Modeller\AppData\Local\Temp\ipykernel_23644\1794173721.py:3: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  _df = _df.pivot_table(index='congestion_category',columns='county',


county,King,Snohomish,Pierce,Kitsap
congestion_category,,,,
Heavy,268,58,72,11
Light,"5,957","2,983","3,583","1,404"
Moderate,433,126,172,22
Severe,38,6,6,1


In [14]:
# Results by Congestion Level
df['speed'] = df['length']/df['auto_time']*60
df['congestion_index'] = df['speed']/df['data2']
df['congestion_index'] = df['congestion_index'].clip(0,1)
df['congestion_category'] = pd.cut(df['congestion_index'], bins=[0,.25,.5,.7,0.99,1], labels=['Severe','Heavy','Moderate','Light','None'])

In [15]:
time_period_list = ['7to8']
_df = df[df['tod'].isin(time_period_list) & (df['@fgts'].isin([1,2]))]
_df = _df.pivot_table(index='congestion_category',columns='county',
               aggfunc='sum',values='length')
_df = _df/len(time_period_list)
_df.to_clipboard()
_df

C:\Users\Modeller\AppData\Local\Temp\ipykernel_23644\1642555342.py:3: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  _df = _df.pivot_table(index='congestion_category',columns='county',


county,King,Snohomish,Pierce,Kitsap
congestion_category,,,,
Severe,11,1,2,0
Heavy,121,24,24,3
Moderate,143,46,55,1
Light,414,179,274,64
None,330,116,167,40


## VMT By Congestion Level

In [16]:
df['Medium and Heavy Trucks'] = df['heavy_truck_vmt']+df['medium_truck_vmt']

_df = df[df['tod'].isin(['5to6','6to7','7to8','8to9'])]
_df = _df.pivot_table(index='congestion_category',columns='county',
               aggfunc='sum',values='Medium and Heavy Trucks')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.loc['Total',:] = _df.sum()
_df

C:\Users\Modeller\AppData\Local\Temp\ipykernel_23644\4017565026.py:4: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  _df = _df.pivot_table(index='congestion_category',columns='county',


,King,Snohomish,Pierce,Kitsap
Light,"397,091","149,556","161,289","26,677"
Moderate,"131,985","24,728","29,502",675
Heavy,"91,641","11,592","8,242","1,210"
Severe,"5,753",449,454,44
Total,"626,471","186,326","199,487","28,606"


In [17]:
_df = df[df['tod'].isin(['15to16','16to17','17to18'])]
_df = _df.pivot_table(index='congestion_category',columns='county',
               aggfunc='sum',values='Medium and Heavy Trucks')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.loc['Total',:] = _df.sum()
_df

C:\Users\Modeller\AppData\Local\Temp\ipykernel_23644\321071178.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  _df = _df.pivot_table(index='congestion_category',columns='county',


,King,Snohomish,Pierce,Kitsap
Light,"254,174","99,429","108,826","18,709"
Moderate,"97,723","18,090","19,106",649
Heavy,"43,819","5,855","4,436",642
Severe,"2,904",304,322,44
Total,"398,621","123,678","132,691","20,044"


## Congestion by County

In [18]:
df['Medium and Heavy Trucks'] = df['heavy_truck_vmt']+df['medium_truck_vmt']

_df = df[df['tod'].isin(['5to6','6to7','7to8','8to9'])]
_df = _df.pivot_table(index='congestion_category',columns='county',
               aggfunc='sum',values='Medium and Heavy Trucks')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.loc['Total',:] = _df.sum()
_df

C:\Users\Modeller\AppData\Local\Temp\ipykernel_23644\4017565026.py:4: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  _df = _df.pivot_table(index='congestion_category',columns='county',


,King,Snohomish,Pierce,Kitsap
Light,"397,091","149,556","161,289","26,677"
Moderate,"131,985","24,728","29,502",675
Heavy,"91,641","11,592","8,242","1,210"
Severe,"5,753",449,454,44
Total,"626,471","186,326","199,487","28,606"


In [19]:
_df = df[df['tod'].isin(['15to16','16to17','17to18'])]
_df = _df.pivot_table(index='congestion_category',columns='county',
               aggfunc='sum',values='Medium and Heavy Trucks')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.loc['Total',:] = _df.sum()
_df

C:\Users\Modeller\AppData\Local\Temp\ipykernel_23644\321071178.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  _df = _df.pivot_table(index='congestion_category',columns='county',


,King,Snohomish,Pierce,Kitsap
Light,"254,174","99,429","108,826","18,709"
Moderate,"97,723","18,090","19,106",649
Heavy,"43,819","5,855","4,436",642
Severe,"2,904",304,322,44
Total,"398,621","123,678","132,691","20,044"


## Congestion on FGTS

In [20]:
pd.options.display.float_format = '{:0,.0f}'.format

# Congested Miles on FGTS versus other Routes
df['Medium and Heavy Trucks'] = df['@mveh']+df['@hveh']

_df = df[df['tod'].isin(['5to6','6to7','7to8','8to9'])]
_df = _df.pivot_table(index='congestion_category',columns='@fgts',
               aggfunc='sum',values='Medium and Heavy Trucks')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.rename(columns={0:'Other Routes', 1: 'T-1', 2: 'T-2'}, inplace=True)
_df_ = _df.copy()
_df.loc['Total',:] = _df.sum()
_df = _df[['T-1','T-2','Other Routes']]
_df

C:\Users\Modeller\AppData\Local\Temp\ipykernel_23644\2204116488.py:7: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  _df = _df.pivot_table(index='congestion_category',columns='@fgts',


,T-1,T-2,Other Routes
Light,"1,659,449","435,103","706,263"
Moderate,"438,267","106,769","212,371"
Heavy,"269,282","93,074","192,064"
Severe,"30,381","35,093","80,672"
Total,"2,397,378","670,039","1,191,370"


In [21]:
pd.options.display.float_format = '{:0,.1%}'.format
_df_/_df_.sum()

,Other Routes,T-1,T-2
Light,59.3%,69.2%,64.9%
Moderate,17.8%,18.3%,15.9%
Heavy,16.1%,11.2%,13.9%
Severe,6.8%,1.3%,5.2%


In [22]:
pd.options.display.float_format = '{:0,.0f}'.format
# Congested Miles on FGTS versus other Routes
df['Medium and Heavy Trucks'] = df['@hveh']+df['@mveh']

_df = df[df['tod'].isin(['15to16','16to17','17to18'])]
_df = _df.pivot_table(index='congestion_category',columns='@fgts',
               aggfunc='sum',values='Medium and Heavy Trucks')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.rename(columns={0:'Other Routes', 1: 'T-1', 2: 'T-2'}, inplace=True)
_df_ = _df.copy()
_df.loc['Total',:] = _df.sum()
_df = _df[['T-1','T-2','Other Routes']]
_df

C:\Users\Modeller\AppData\Local\Temp\ipykernel_23644\907623602.py:6: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  _df = _df.pivot_table(index='congestion_category',columns='@fgts',


,T-1,T-2,Other Routes
Light,"1,077,966","304,156","456,953"
Moderate,"303,472","61,004","129,794"
Heavy,"142,093","51,899","117,574"
Severe,"18,351","23,855","48,574"
Total,"1,541,882","440,914","752,895"


In [23]:
pd.options.display.float_format = '{:0,.1%}'.format
_df_/_df_.sum()

,Other Routes,T-1,T-2
Light,60.7%,69.9%,69.0%
Moderate,17.2%,19.7%,13.8%
Heavy,15.6%,9.2%,11.8%
Severe,6.5%,1.2%,5.4%


## Equity

### Households Within 500' of Heavy Truck Volumes

Total number of households within 500' of T-1 and T-2 routes

- T-1: More than 10 million tons per year

- T-2: 2 4 million to 10 million tons per year

In [24]:
import polars as pl

# Intersect buffer with land use file
df_lu = pl.read_csv(r'..\..\..\..\outputs\landuse\parcels_urbansim.txt', separator=' ').to_pandas()

# Load as a geodataframe
gdf_lu = gpd.GeoDataFrame(
    df_lu, geometry=gpd.points_from_xy(df_lu.xcoord_p, df_lu.ycoord_p))
# Set the coordinate reference system (CRS) to match the land use data
gdf_lu.crs = 'EPSG:2285'

# Buffer the parcels at 500ft
gdf_lu['geometry'] = gdf_lu.buffer(500)

In [25]:

# Intersect this geography  with the network shapefile
gdf_network = gpd.read_file(r'..\..\..\..\inputs\scenario\networks\shapefiles\AM\AM_edges.shp')
# Do not include connectors since these are abstracted ul3==5; also remove weave links ul3==0 
gdf_network = gdf_network[~gdf_network.ul3.isin([0,5])]
# Truck network links are those that are in FGTS 1 or 2 system
gdf_network = gdf_network[gdf_network['FGTS'].isin([1,2])]

gdf_intersect = gpd.overlay(gdf_network, gdf_lu, how="intersection", keep_geom_type=False)

# Will need to relaculate the lengths since some were split across the regional geographies
gdf_intersect['new_length'] = gdf_intersect.geometry.length/5280.0

# filter out the polygon results and only keep lines
gdf_intersect = gdf_intersect[gdf_intersect.geometry.type.isin(['MultiLineString','LineString'])]

In [26]:
truck_parcels = gdf_intersect.groupby('parcelid').first()[['hh_p']].reset_index()


# Result should be the network components with some flags for the parcelid
# We can take the parcel information, join with parcel info and group
# from input_configuration import base_year
import toml
config = toml.load(os.path.join(os.getcwd(),r'../../../../configuration/input_configuration.toml'))

parcel_geog = pd.read_sql_table('parcel_'+config['base_year']+'_geography', 'sqlite:///../../../../inputs/db/'+config['db_name'])
df = truck_parcels.merge(parcel_geog,left_on='parcelid', right_on='ParcelID')

In [27]:
# Get the total number of households that in equtiy geograhpies
# Comprae the percent of those that are in the buffer versus those that are not
# For the 4 equity groups, perform the calc and add as a table
results_df = pd.DataFrame()
for col, name in {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                        'equity_focus_areas_2023__efa_dis': 'Disability',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
                  }.items():
    _df = df[[col,'hh_p']].groupby(col).sum()[['hh_p']]
    _df['equity_group'] = name
    results_df = pd.concat([results_df,_df])
results_df = results_df.reset_index()
results_df = results_df[results_df['index']>=0]

### Total Households Within 500' of T-1/T-2 Routes by Equity Group

In [28]:
_df = results_df.pivot_table(index='index', columns='equity_group', values='hh_p', aggfunc='sum')
_df.index = _df.index.astype('int').map({0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'}
                                )
pd.options.display.float_format = '{:0,.0f}'.format
_df.loc['Region',:] = _df.sum(axis=0)
_df.astype('float')

equity_group,Disability,LEP,Older,People of Color,Poverty,Youth
index,,,,,,
Below Regional Average,"96,729","105,618","118,343","75,091","91,169","124,528"
Above Regional Average,"56,908","41,623","53,783","65,912","56,922","45,724"
Higher Share of Equity Population,"33,259","39,655","14,770","45,893","38,805","16,644"
Region,"186,896","186,896","186,896","186,896","186,896","186,896"


In [29]:
pd.options.display.float_format = '{:0,.1%}'.format
_df.drop('Region')/_df.drop('Region').sum()

equity_group,Disability,LEP,Older,People of Color,Poverty,Youth
index,,,,,,
Below Regional Average,51.8%,56.5%,63.3%,40.2%,48.8%,66.6%
Above Regional Average,30.4%,22.3%,28.8%,35.3%,30.5%,24.5%
Higher Share of Equity Population,17.8%,21.2%,7.9%,24.6%,20.8%,8.9%


In [30]:
_df = results_df[["equity_group","hh_p","index"]].pivot_table(index='equity_group',columns='index',values='hh_p')
_df.rename(columns={0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'}, inplace=True)
_df.index.name = None
_df.columns.name = None
_df['Total Households'] = _df.sum(axis=1)
_df_buffer = _df.copy()

df_lu_tot = df_lu[['parcelid','hh_p']].merge(parcel_geog,left_on='parcelid', right_on='ParcelID')
df_lu_tot['Region'] = 1
results_df_tot = pd.DataFrame()
for col, name in {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                        'equity_focus_areas_2023__efa_dis': 'Disability',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
                  'Region': 'Region'
                  }.items():
    _df = df_lu_tot[['hh_p',col]].groupby(col).sum()[['hh_p']]
    _df['equity_group'] = name
    results_df_tot = pd.concat([results_df_tot, _df])
results_df_tot = results_df_tot.reset_index()
pd.options.display.float_format = '{:0,.0f}'.format

_df = results_df_tot.pivot_table(index='equity_group',columns='index',values='hh_p')
_df.rename(columns={0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'}, inplace=True)
_df.index.name = None
_df.columns.name = None
_df['Total Households'] = _df.sum(axis=1)
_df_tot = _df.copy()
_df = _df_tot.merge(_df_buffer, left_index=True, right_index=True, suffixes=['_tot','_buffer'])

pd.options.display.float_format = '{:0,.1%}'.format
efa_list = ['Below Regional Average', 'Above Regional Average','Higher Share of Equity Population', 'Total Households']
for efa in efa_list:
    _df[efa] = _df[f'{efa}_buffer']/_df[f'{efa}_tot']
_df[efa_list].T


,Disability,LEP,Older,People of Color,Poverty,Youth
Below Regional Average,10.3%,9.7%,13.0%,8.0%,8.8%,13.1%
Above Regional Average,10.8%,11.7%,9.1%,13.2%,12.8%,8.1%
Higher Share of Equity Population,12.5%,13.8%,6.3%,15.7%,15.6%,7.7%
Total Households,10.8%,10.8%,10.8%,10.8%,10.8%,10.8%
